In [0]:
# Install cloudgenix SDK in Runtime (may need restart - will prompt). Can add add'l lines for any other non-default modules you need (pandas, etc)
!pip install --upgrade cgx_notebooks


In [0]:
# Import and basic setup.
from cgx_notebooks.aaron import *

# login and start SDK
sdk = instantiate()
# ask for auth_token
ask_for_auth_token_and_login(sdk)
# build caches
idname, all_id2n, sites_n2id, elements_n2id, apps_n2id = create_api_maps_cache(sdk)
# set apps_list for app list collection
apps_list = []



In [0]:
# Get Site from form Based on name 
#@title Enter or change Site Name
site_name = "Branch1" #@param {type:"string"}
site_id = select_site(site_name, sites_n2id)

In [0]:
# Get element from form Based on name 
#@title Enter or change Element Name
element_name = "BR1-1" #@param {type:"string"}
element_id = select_element(element_name, elements_n2id)

In [0]:
# Get app from form Based on name. You can run this multiple times to select multiple apps.
# Set apps_list = [] to reset list.
#@title Enter App Name
app_name = "salesforce" #@param {type:"string"}
select_app(app_name, apps_list, apps_n2id)

In [0]:
#@title Select Timeframe
now_or_specific = "Now" #@param ["Now", "Enter Specific Time(UTC)"]
enter_specific_date = "2020-03-03" #@param {type:"date"}
specific_hour = 12 #@param {type:"slider", min:0, max:24, step:1}
specific_minute = 0 #@param {type:"slider", min:0, max:59, step:1}

# Get time window
import datetime

if now_or_specific == "Enter Specific Time(UTC)":
  now = datetime.datetime.strptime(f"{enter_specific_date}T{specific_hour}:{specific_minute}:00.000Z",
                                   "%Y-%m-%dT%H:%M:%S.%fZ")
else:
  now = datetime.datetime.utcnow()

# compute 1h delta
end_time = now.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
start = now - datetime.timedelta(hours=1)
start_time = start.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

# start_time = "2020-02-26T19:52:45.721Z"
# end_time = "2020-02-26T20:52:45.721Z"
print(f"Start Time: {start_time}\nEnd Time: {end_time}")

In [46]:
# Flow relationship graph.

# get dataframe (can limit - example, source_ips="10.1.100.19/32")
df = query_flows_to_df(sdk, all_id2n, start_time, end_time, site_id, app_id_list=apps_list, destination_ips=None,
                       destination_ports=None, protocol=None, source_ips=None, source_ports=None)

# create chart
height = 1700
width = 1500
# create sankey. 
# cat_cols = left, middle, right data values. value_cols = addititve values. 'priority_class', 'traffic_type'
#'row_count' is specialmeans use # of matching rows, but can do any column - say retransmit_pkts_c2s, or 'unknown_domain'. 
fig, fig_df = generate_flow_app_sankey(df, cat_cols=['source_ip', 'app_id', 'destination_ip'], value_cols='row_count', title=f'Flow relations for site {site_name}',height=height, width=width, thickness=20, 
                                       pad=10, line=None, return_df=True)

# set the display area and render the chart
display(Javascript(data="google.colab.output.setIframeHeight(0, true, {{maxHeight: {0}}})".format(height)))
fig.show()


<IPython.core.display.Javascript object>

In [0]:
# debug print dataframe
display(df[0:10])

In [0]:
# reset apps
apps_list =[]